###  Module & Utility Imports

In [13]:
import pandas as pd
import numpy as np

### Comparison between the main model and benchmark models (Main manuscript)

In [14]:
Prefix_map = { 'ConVAE' : 'C-VAE',
               'FACVAE' : 'FAC-VAE',
              'TCVAE' : 'TC-VAE',
              'VDVAE' : 'VD-VAE'}


# Load original dataframe
BenchAnalAccMItable = pd.read_csv('./EvalResults/SummaryTables/BenchAnalAccMItable.csv')
BenchAnalAccMItable['Type'] = BenchAnalAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Pivot table with 'ISCOREScal' values
PivotTable = BenchAnalAccMItable.pivot_table(
    index=['Source', 'Type'],
    columns='Prefix',
    values='ISCOREScal'
).reset_index()

# Rename 'SKZFC' to 'Ours' and move to last column
PivotTable = PivotTable.rename(columns={'SKZFC': 'Ours'})
cols = [col for col in PivotTable.columns if col not in ['Ours']] + ['Ours']
PivotTable = PivotTable[cols]

# Identify model columns (exclude 'Source' and 'Type')
model_cols = [col for col in PivotTable.columns if col not in ['Source', 'Type']]

# Bold the max ISCOREScal in each row
def bold_max_in_row(row):
    values = row[model_cols].astype(float)
    max_val = values.max()
    return [
        f"\\textbf{{{v:.4f}}}" if np.isclose(v, max_val) else f"{v:.4f}"
        for v in values
    ]

# Apply bold formatting row-wise
formatted_model_values = PivotTable[model_cols].apply(lambda row: bold_max_in_row(row), axis=1, result_type='expand')
formatted_model_values.columns = model_cols

# Combine with Source and Type
formatted_table = pd.concat([PivotTable[['Source', 'Type']], formatted_model_values], axis=1)
formatted_table = formatted_table.rename(columns=Prefix_map).copy()

# Generate LaTeX
latex_table_code = r"""\begin{table}[t]
\centering
\caption{Table of benchmark comparisons focused on ISCORE only.}
\label{tab:iscore_comparison}
""" + formatted_table.to_latex(
    index=False,
    escape=False,
    float_format="%.4f",
    column_format="ll|" + "r"*(len(formatted_table.columns)-2),
    bold_rows=False
) + r"""\end{table}"""

# Output LaTeX
print(latex_table_code)


\begin{table}[t]
\centering
\caption{Table of benchmark comparisons focused on ISCORE only.}
\label{tab:iscore_comparison}
\begin{tabular}{ll|rrrrrrrr}
\toprule
Source & Type & C-VAE & DiffWave & FAC-VAE & TC-VAE & VD-VAE & VDWave & Wavenet & Ours \\
\midrule
Mimic & ABP & 0.2027 & 0.3337 & 0.1963 & 0.1199 & 0.1303 & 0.3338 & 0.2356 & \textbf{2.0832} \\
Mimic & ECG & 0.5653 & 0.6366 & 0.2435 & 0.5838 & 0.2548 & 0.6374 & 0.2583 & \textbf{2.4422} \\
VitalDB & ABP & 0.1528 & 0.3419 & 0.1604 & 0.1867 & 0.1369 & 0.3420 & 0.1804 & \textbf{1.9574} \\
VitalDB & ECG & 0.4400 & 0.6585 & 0.4000 & 0.5647 & 0.3364 & 0.6590 & 0.4718 & \textbf{2.0179} \\
\bottomrule
\end{tabular}
\end{table}


### Comparison between the main model and benchmark models (Supplementary file)

In [16]:
# Check the actual column names
actual_columns = BenchAnalAccMItable.columns.tolist()

# Define major metrics based on actual column names
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]

type_name_map = {
    'ART': 'ABP',
    'II': 'ECG'
}


# Get unique values for Source and Type columns
sources = BenchAnalAccMItable['Source'].unique()
types = BenchAnalAccMItable['Type'].unique()

# Rename 'SKZFC' to 'Ours' for easier interpretation
BenchAnalAccMItable.loc[BenchAnalAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Initialize a dictionary to store the generated LaTeX tables
split_tables_latex = {}

# Generate LaTeX table for each (Source, Type) pair
for source in sources:
    for type_ in types:
        subset = BenchAnalAccMItable[
            (BenchAnalAccMItable['Source'] == source) & 
            (BenchAnalAccMItable['Type'] == type_)
        ]
        
        table = subset.pivot_table(index='Prefix', values=major_metrics)
        table = table.reset_index()
        table = table[['Prefix'] + major_metrics].copy()
        
        # Rename columns
        col_map = {
            'Prefix': 'Model', 
            '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
            '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
            '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$', 
            'FQI': 'FQI $\\downarrow$',
            'RMSE': 'RMSE $\\downarrow$',
            'ISCOREScal': 'ISCORE $\\uparrow$'
        }
        table = table.rename(columns=col_map)
        
        # Move Ours to last
        if 'Ours' in table['Model'].values:
            ours_row = table[table['Model'] == 'Ours']
            table = table[table['Model'] != 'Ours']
            table = pd.concat([table, ours_row], ignore_index=True)

        # Bold max values for ↑ metrics and min values for ↓ metrics
        metrics_up = ['(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
                      '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
                      'ISCORE $\\uparrow$']
        metrics_down = ['(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
                        'FQI $\\downarrow$', 'RMSE $\\downarrow$']

        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, max_val) else f"{x:.4f}")

        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, min_val) else f"{x:.4f}")

        # Generate LaTeX
        latex_code = r"""\begin{table}[h]
\centering
\caption{Benchmark comparisons for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format="c|" + "c"*(len(table.columns)-2)+'|c' )  + r"""\end{table}"""

        split_tables_latex[(source, type_)] = latex_code


# Example: print one latex table
print(split_tables_latex[('Mimic', 'ABP')])


\begin{table}[h]
\centering
\caption{Benchmark comparisons for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
ConVAE & 0.0519 & 0.9339 & 0.2500 & 0.0980 & 3.1147 & 0.2027 \\
DiffWave & 0.0000 & 0.0018 & 0.0005 & 0.0439 & 5.0266 & 0.3337 \\
FACVAE & 0.0609 & 0.9601 & 0.2057 & 0.0885 & 2.9360 & 0.1963 \\
TCVAE & 0.1286 & 1.5965 & 0.2364 & 0.1085 & 3.2440 & 0.1199 \\
VDVAE & 0.0655 & 1.3576 & 0.1124 & 0.0890 & \textbf{2.8555} & 0.1303 \\
VDWave & 0.0000 & \textbf{0.0014} & 0.0004 & \textbf{0.0437} & 5.0093 & 0.3338 \\
Wavenet & 0.0000 & 0.4754 & 0.0487 & 0.0747 & 4.6166 & 0.2356 \\
Ours & \textbf{3.6300} & 0.0478 & \textbf{0.8360} & 0.0883 & 3.6893 & \textbf{2.0832} \\
\bottomrule
\end{tabular}
\end{table}


In [17]:
print(split_tables_latex[('Mimic', 'ABP')])
print()
print(split_tables_latex[('Mimic', 'ECG')])
print()
print(split_tables_latex[('VitalDB', 'ABP')])
print()
print(split_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Benchmark comparisons for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
ConVAE & 0.0519 & 0.9339 & 0.2500 & 0.0980 & 3.1147 & 0.2027 \\
DiffWave & 0.0000 & 0.0018 & 0.0005 & 0.0439 & 5.0266 & 0.3337 \\
FACVAE & 0.0609 & 0.9601 & 0.2057 & 0.0885 & 2.9360 & 0.1963 \\
TCVAE & 0.1286 & 1.5965 & 0.2364 & 0.1085 & 3.2440 & 0.1199 \\
VDVAE & 0.0655 & 1.3576 & 0.1124 & 0.0890 & \textbf{2.8555} & 0.1303 \\
VDWave & 0.0000 & \textbf{0.0014} & 0.0004 & \textbf{0.0437} & 5.0093 & 0.3338 \\
Wavenet & 0.0000 & 0.4754 & 0.0487 & 0.0747 & 4.6166 & 0.2356 \\
Ours & \textbf{3.6300} & 0.0478 & \textbf{0.8360} & 0.0883 & 3.6893 & \textbf{2.0832} \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\center

In [18]:
# Re-load the original dataframe
SenseAccMItable = pd.read_csv('./EvalResults/SummaryTables/SenseAccMItable.csv')
SenseAccMItable['Type'] = SenseAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Define metric categories and mappings
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]
col_map = {
    '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
    'FQI': 'FQI $\\downarrow$',
    'RMSE': 'RMSE $\\downarrow$',
    'ISCOREScal': 'ISCORE $\\uparrow$'
}

metrics_up = [col_map[k] for k in [major_metrics[0], major_metrics[2], major_metrics[5]]]
metrics_down = [col_map[k] for k in [major_metrics[1], major_metrics[3], major_metrics[4]]]

type_name_map = {'ART': 'ABP', 'II': 'ECG'}
structural_info = ['Depth', 'LatDim', 'Comp']

# Clean model naming
SenseAccMItable.loc[SenseAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Prepare output
split_sense_tables_latex = {}

# Iterate per Source × Type
for source in SenseAccMItable['Source'].unique():
    for type_ in SenseAccMItable['Type'].unique():
        subset = SenseAccMItable[
            (SenseAccMItable['Source'] == source) &
            (SenseAccMItable['Type'] == type_)
        ]

        # Prepare and rename
        table = subset[structural_info + major_metrics].copy().reset_index(drop=True)
        table = table.rename(columns=col_map)

        # Ensure numeric for comparison
        for col in metrics_up + metrics_down:
            table[col] = pd.to_numeric(table[col], errors='coerce')

        # Format and bold
        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, max_val, atol=1e-8) else f"{x:.4f}")
        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, min_val, atol=1e-8) else f"{x:.4f}")

        table = table.rename(columns={'Depth':'$\zeta$', 'LatDim':'$J$', 'Comp':'$C$'})
        # LaTeX generation with vertical line after last structural column
        split_idx = len(structural_info)
        column_format = "c" * split_idx + "|" + "c" * (len(table.columns) - split_idx-1) +'|c'

        latex_code = r"""\begin{table}[h]
\centering
\caption{Sensitivity analysis for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:sense_result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format=column_format) + r"""\end{table}"""

        split_sense_tables_latex[(source, type_)] = latex_code

# Example output
print(split_sense_tables_latex[('Mimic', 'ECG')])

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ECG on the Mimic}
\label{tab:sense_result_mimic_ecg}
\begin{tabular}{ccc|ccccc|c}
\toprule
$\zeta$ & $J$ & $C$ & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
1 & 30 & 500 & \textbf{3.2754} & 0.1578 & 0.7273 & 0.2253 & 0.4013 & \textbf{2.8371} \\
1 & 30 & 800 & 2.6602 & 0.2024 & \textbf{0.7955} & \textbf{0.2227} & 0.3802 & 2.4422 \\
1 & 50 & 800 & 2.7671 & 0.1610 & 0.7656 & 0.2239 & 0.3808 & 2.5246 \\
2 & 30 & 500 & 1.3212 & \textbf{0.0053} & 0.5006 & 1.0119 & \textbf{0.3796} & 1.0304 \\
2 & 30 & 800 & 3.1528 & 0.2158 & 0.4385 & 0.2314 & 0.3870 & 2.5101 \\
2 & 50 & 800 & 3.0235 & 0.2510 & 0.6106 & 0.2943 & 0.3830 & 2.4375 \\
\bottomrule
\end{tabular}
\end{table}


In [19]:
print(split_sense_tables_latex[('Mimic', 'ABP')])
print()
print(split_sense_tables_latex[('Mimic', 'ECG')])
print()
print(split_sense_tables_latex[('VitalDB', 'ABP')])
print()
print(split_sense_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ABP on the Mimic}
\label{tab:sense_result_mimic_abp}
\begin{tabular}{ccc|ccccc|c}
\toprule
$\zeta$ & $J$ & $C$ & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
1 & 30 & 500 & \textbf{3.9716} & 0.0554 & \textbf{0.8931} & 0.0775 & 5.1116 & \textbf{2.2437} \\
1 & 30 & 800 & 3.6300 & \textbf{0.0478} & 0.8360 & 0.0883 & 3.6893 & 2.0832 \\
1 & 50 & 800 & 3.5172 & 0.0572 & 0.7897 & 0.0860 & 3.6732 & 2.0104 \\
2 & 30 & 500 & 3.3894 & 0.1425 & 0.4871 & \textbf{0.0657} & 4.9872 & 1.7721 \\
2 & 30 & 800 & 3.7157 & 0.0635 & 0.4233 & 0.1079 & 3.2612 & 1.9154 \\
2 & 50 & 800 & 3.6028 & 0.1089 & 0.3448 & 0.0722 & \textbf{3.2587} & 1.8698 \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Sensitivity analysis for ECG on the Mimic}
\label{tab:sen

In [22]:
# Re-load the original dataframe
AblaAccMItable = pd.read_csv('./EvalResults/SummaryTables/AblaAccMItable.csv')
AblaAccMItable['Type'] = AblaAccMItable['Type'].replace({'ART': 'ABP', 'II': 'ECG'})

# Check the actual column names
actual_columns = AblaAccMItable.columns.tolist()

# Define major metrics based on actual column names
major_metrics = [
    '(i) $I(V; \\acute{Z} \\mid Z)$',
    '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$',
    '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$',
    'FQI', 'RMSE', 'ISCOREScal'
]

type_name_map = {
    'ART': 'ABP',
    'II': 'ECG'
}


# Get unique values for Source and Type columns
sources = AblaAccMItable['Source'].unique()
types = AblaAccMItable['Type'].unique()

# Rename 'SKZFC' to 'Ours' for easier interpretation
AblaAccMItable.loc[AblaAccMItable['Prefix'] == 'SKZFC', 'Prefix'] = 'Ours'

# Initialize a dictionary to store the generated LaTeX tables
split_tables_latex = {}

# Generate LaTeX table for each (Source, Type) pair
for source in sources:
    for type_ in types:
        subset = AblaAccMItable[
            (AblaAccMItable['Source'] == source) & 
            (AblaAccMItable['Type'] == type_)
        ]
        
        table = subset.pivot_table(index='Prefix', values=major_metrics)
        table = table.reset_index()
        table = table[['Prefix'] + major_metrics].copy()
        
        # Rename columns
        col_map = {
            'Prefix': 'Model', 
            '(i) $I(V; \\acute{Z} \\mid Z)$': '(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
            '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z})$': '(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
            '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z})$': '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$', 
            'FQI': 'FQI $\\downarrow$',
            'RMSE': 'RMSE $\\downarrow$',
            'ISCOREScal': 'ISCORE $\\uparrow$'
        }
        table = table.rename(columns=col_map)
        
        # Move Ours to last
        if 'Ours' in table['Model'].values:
            ours_row = table[table['Model'] == 'Ours']
            table = table[table['Model'] != 'Ours']
            table = pd.concat([table, ours_row], ignore_index=True)

        # Bold max values for ↑ metrics and min values for ↓ metrics
        metrics_up = ['(i) $I(V; \\acute{Z} \\mid Z) \\uparrow$',
                      '(iii) $I(S;\\acute{\\Theta} \\mid \\acute{Z}) \\uparrow$',
                      'ISCORE $\\uparrow$']
        metrics_down = ['(ii) $I(V;\\acute{\\Theta} \\mid \\acute{Z}) \\downarrow$',
                        'FQI $\\downarrow$', 'RMSE $\\downarrow$']

        for col in metrics_up:
            max_val = table[col].max()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, max_val) else f"{x:.4f}")

        for col in metrics_down:
            min_val = table[col].min()
            table[col] = table[col].apply(lambda x: f"\\textbf{{{x:.4f}}}" if np.isclose(x, min_val) else f"{x:.4f}")

        # Generate LaTeX
        latex_code = r"""\begin{table}[h]
\centering
\caption{Ablation studies for """ + f"{type_name_map.get(type_, type_)}" + r""" on the """ + f"{source}" + r"""}
\label{tab:result_""" + f"{source.lower()}_{type_.lower()}" + r"""}
""" + table.to_latex(index=False, escape=False, column_format="c|" + "c"*(len(table.columns)-2)+'|c' )  + r"""\end{table}"""

        split_tables_latex[(source, type_)] = latex_code


# Example: print one latex table
print(split_tables_latex[('Mimic', 'ABP')])


\begin{table}[h]
\centering
\caption{Ablation studies for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
FC & 3.2819 & 0.9896 & 0.3601 & 0.0879 & \textbf{3.6565} & 1.2810 \\
SKZ & 3.0829 & \textbf{0.0000} & \textbf{0.8838} & \textbf{0.0786} & 5.8453 & 1.8479 \\
Ours & \textbf{3.5172} & 0.0572 & 0.7897 & 0.0860 & 3.6732 & \textbf{2.0104} \\
\bottomrule
\end{tabular}
\end{table}


In [23]:
AblaAccMItable

,Model,(i) $I(V; \acute{Z} \mid Z)$,(ii) $I(V;\acute{\Theta} \mid \acute{Z})$,(iii) $I(S;\acute{\Theta} \mid \acute{Z})$,FQI,RMSE,Prefix,Type,Depth,LatDim,Comp,Source,ISCORE,Scaling,ISCOREScal
0,FC_ART_1_50_800_Mimic,3.281942,9.895840e-01,0.360087,0.087858,3.656514,FC,ABP,1,50,800,Mimic,2.720551,0.470857,1.280990
1,FC_ART_1_50_800_VitalDB,3.881827,2.798609e-01,0.743773,0.044483,4.591867,FC,ABP,1,50,800,VitalDB,4.358617,0.483313,2.106576
2,FC_II_1_50_800_Mimic,2.850989,3.104209e-01,0.672421,0.211902,0.388613,FC,ECG,1,50,800,Mimic,3.252437,0.743520,2.418253
3,FC_II_1_50_800_VitalDB,2.023524,2.557765e-01,0.712910,0.176594,0.048588,FC,ECG,1,50,800,VitalDB,2.561028,0.895346,2.293007
4,SKZFC_ART_1_50_800_Mimic,3.517190,5.724298e-02,0.789660,0.085957,3.673249,Ours,ABP,1,50,800,Mimic,4.263776,0.471514,2.010429
5,SKZFC_ART_1_50_800_VitalDB,4.272571,1.886027e-02,1.000776,0.042677,4.600318,Ours,ABP,1,50,800,VitalDB,5.259697,0.484135,2.546401
6,SKZFC_II_1_50_800_Mimic,2.767104,1.610470e-01,0.765604,0.223932,0.380783,Ours,ECG,1,50,800,Mimic,3.405418,0.741348,2.524598
7,SKZFC_II_1_50_800_VitalDB,2.004492,1.016334e-01,0.691140,0.169844,0.049781,Ours,ECG,1,50,800,VitalDB,2.666059,0.897617,2.393101
8,SKZ_ART_1_50_800_Mimic,3.082947,7.704943e-08,0.883813,0.078589,5.845308,SKZ,ABP,1,50,800,Mimic,3.985518,0.463657,1.847913
9,SKZ_ART_1_50_800_VitalDB,1.296760,1.233494e-11,0.860050,0.243360,5.200905,SKZ,ABP,1,50,800,VitalDB,2.266286,0.394750,0.894617


In [24]:
print(split_tables_latex[('Mimic', 'ABP')])
print()
print(split_tables_latex[('Mimic', 'ECG')])
print()
print(split_tables_latex[('VitalDB', 'ABP')])
print()
print(split_tables_latex[('VitalDB', 'ECG')])

\begin{table}[h]
\centering
\caption{Ablation studies for ABP on the Mimic}
\label{tab:result_mimic_abp}
\begin{tabular}{c|ccccc|c}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\
\midrule
FC & 3.2819 & 0.9896 & 0.3601 & 0.0879 & \textbf{3.6565} & 1.2810 \\
SKZ & 3.0829 & \textbf{0.0000} & \textbf{0.8838} & \textbf{0.0786} & 5.8453 & 1.8479 \\
Ours & \textbf{3.5172} & 0.0572 & 0.7897 & 0.0860 & 3.6732 & \textbf{2.0104} \\
\bottomrule
\end{tabular}
\end{table}

\begin{table}[h]
\centering
\caption{Ablation studies for ECG on the Mimic}
\label{tab:result_mimic_ecg}
\begin{tabular}{c|ccccc|c}
\toprule
Model & (i) $I(V; \acute{Z} \mid Z) \uparrow$ & (ii) $I(V;\acute{\Theta} \mid \acute{Z}) \downarrow$ & (iii) $I(S;\acute{\Theta} \mid \acute{Z}) \uparrow$ & FQI $\downarrow$ & RMSE $\downarrow$ & ISCORE $\uparrow$ \\